In [1]:
%load_ext autoreload
%autoreload 2

---

In [2]:
from src.data.nitter_scraper_standalone_v2 import Tweet, TweetScraper

tweets = TweetScraper.load_collected_tweets(path='../.data/data.pkl')

In [3]:
import pandas as pd

tweet_dicts =  [{"url": tweet.url, "date": tweet.date, "rawContent": tweet.rawContent} for tweet in tweets]

df = pd.DataFrame(tweet_dicts)
df

,url,date,rawContent
0,https://twitter.com/_Bob_S/status/164159110508...,"Mar 30, 2023 · 11:59 PM UTC",Govt IT security isnt 'a nice thing to do': it...
1,https://twitter.com/WYSIWYGVentures/status/164...,"Mar 30, 2023 · 11:59 PM UTC",ISC West 2023: Cyberattackers Are Targeting Ph...
2,https://twitter.com/HackerAran7/status/1641591...,"Mar 30, 2023 · 11:59 PM UTC",What’s the hack. #stem #science #stemeducation...
3,https://twitter.com/bytefeedai/status/16415909...,"Mar 30, 2023 · 11:59 PM UTC",BuzzFeed Is Using AI To Write SEO-Bait Travel ...
4,https://twitter.com/bytefeedai/status/16415909...,"Mar 30, 2023 · 11:59 PM UTC",Tech Leaders Sign Open Letter Calling for Paus...
...,...,...,...
995005,https://twitter.com/Imsundar_b/status/15760518...,"Oct 1, 2022 · 3:30 AM UTC",Caris' Precision Oncology Alliance Welcomes St...
995006,https://twitter.com/DevOpsIS/status/1576051840...,"Oct 1, 2022 · 3:30 AM UTC",According to the New Ways of Working Survey 20...
995007,https://twitter.com/NAJobsCognizant/status/157...,"Oct 1, 2022 · 3:30 AM UTC",Learn how you can influence #digital innovatio...
995008,https://twitter.com/BuzzInshort/status/1576051...,"Oct 1, 2022 · 3:30 AM UTC",⚡Top 10 Technology News Stories This Morning a...


In [4]:
duplicates = df['rawContent'].duplicated()
num_duplicates = duplicates.sum()
print(num_duplicates)

140772


In [5]:
from langdetect import detect
from tqdm import tqdm

test_df = df.head(50_000)

def detect_language(text):
    try:
        lang = detect(text)
    except:
        lang = None
    return lang

tqdm.pandas()
test_df['lang'] = test_df['rawContent'].progress_apply(detect_language)

100%|███████████████████████████████████████████████████████████████████████████| 50000/50000 [03:01<00:00, 275.03it/s]

                                                     url   
0      https://twitter.com/_Bob_S/status/164159110508...  \
1      https://twitter.com/WYSIWYGVentures/status/164...   
2      https://twitter.com/HackerAran7/status/1641591...   
3      https://twitter.com/bytefeedai/status/16415909...   
4      https://twitter.com/bytefeedai/status/16415909...   
...                                                  ...   
49995  https://twitter.com/ProjectConsult5/status/163...   
49996  https://twitter.com/DaveHatter/status/16382474...   
49997  https://twitter.com/GeeksTechTweets/status/163...   
49998  https://twitter.com/javascriptflx/status/16382...   
49999  https://twitter.com/GeeksTechTweets/status/163...   

                              date   
0      Mar 30, 2023 · 11:59 PM UTC  \
1      Mar 30, 2023 · 11:59 PM UTC   
2      Mar 30, 2023 · 11:59 PM UTC   
3      Mar 30, 2023 · 11:59 PM UTC   
4      Mar 30, 2023 · 11:59 PM UTC   
...                            ...   
49995   Mar 2


C:\Users\lukas\AppData\Local\Temp\ipykernel_19296\1335496866.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['lang'] = test_df['rawContent'].progress_apply(detect_language)


In [6]:
test_df

,url,date,rawContent,lang
0,https://twitter.com/_Bob_S/status/164159110508...,"Mar 30, 2023 · 11:59 PM UTC",Govt IT security isnt 'a nice thing to do': it...,en
1,https://twitter.com/WYSIWYGVentures/status/164...,"Mar 30, 2023 · 11:59 PM UTC",ISC West 2023: Cyberattackers Are Targeting Ph...,en
2,https://twitter.com/HackerAran7/status/1641591...,"Mar 30, 2023 · 11:59 PM UTC",What’s the hack. #stem #science #stemeducation...,en
3,https://twitter.com/bytefeedai/status/16415909...,"Mar 30, 2023 · 11:59 PM UTC",BuzzFeed Is Using AI To Write SEO-Bait Travel ...,en
4,https://twitter.com/bytefeedai/status/16415909...,"Mar 30, 2023 · 11:59 PM UTC",Tech Leaders Sign Open Letter Calling for Paus...,en
...,...,...,...,...
49995,https://twitter.com/ProjectConsult5/status/163...,"Mar 21, 2023 · 6:33 PM UTC",The potential for AI and robotics to revolutio...,en
49996,https://twitter.com/DaveHatter/status/16382474...,"Mar 21, 2023 · 6:33 PM UTC","Tune in to #700WLW today at 4:05 PM, I'll be o...",en
49997,https://twitter.com/GeeksTechTweets/status/163...,"Mar 21, 2023 · 6:33 PM UTC","Google Opens Access to Bard AI Chatbot, Racing...",en
49998,https://twitter.com/javascriptflx/status/16382...,"Mar 21, 2023 · 6:33 PM UTC",Functional Programming Principles in Scala Cou...,en


In [7]:
non_english_posts = test_df.query('lang != "en"')
non_english_posts

,url,date,rawContent,lang
228,https://twitter.com/bluebay700/status/16415781...,"Mar 30, 2023 · 11:08 PM UTC",#Tech \n...uhem....,de
950,https://twitter.com/erikmackenzie/status/16415...,"Mar 30, 2023 · 9:06 PM UTC",#Imagine being #anti #innovation 🤦🏻‍♂️,it
1023,https://twitter.com/kuriharan/status/164154584...,"Mar 30, 2023 · 9:00 PM UTC",Learn it. The Nostr Privacy Paradox bitcoinmag...,it
1746,https://twitter.com/humanwareonline/status/164...,"Mar 30, 2023 · 7:15 PM UTC",Potere di un project manager | Innovation & Pr...,hr
2088,https://twitter.com/f_bruneault/status/1641509...,"Mar 30, 2023 · 6:35 PM UTC",The Open Letter to Stop 'Dangerous' #AI Race I...,fr
...,...,...,...,...
46799,https://twitter.com/SDTech_Groupe/status/16385...,"Mar 22, 2023 · 3:00 PM UTC",Picture lance son programme The Circular\nbit....,fr
46911,https://twitter.com/Newsnext5/status/163855334...,"Mar 22, 2023 · 2:48 PM UTC",Aadhaar Card Update: Do your Aadhaar document ...,nl
48033,https://twitter.com/oshonubi/status/1638326783...,"Mar 21, 2023 · 11:48 PM UTC","#repost Ioannis Kapranos, re: job openings. #e...",nl
49633,https://twitter.com/asegun_/status/16382578189...,"Mar 21, 2023 · 7:14 PM UTC",Some dark-mode goodness\n\n#uidesign #uxdesign...,fr


---

In [8]:
from src.data.nitter_scraper_standalone_v2 import Tweet, TweetScraper

tweets = TweetScraper.load_collected_tweets(path='../.data/rdata.pkl')
print(len(tweets))

19398
